# Notebook to process AWS Application Discovery Output

## 1. Parameter definition

This notebook performs joining and data processing of data provided by the AWS ADS service. We join the following files:

1. EC2 Instance Recommendations
2. System Performance

The insights this Notebook provides are:

1. EC2 cost estimation
2. EBS storage estimation
3. DTO cost estimation, assuming that 20% of the written traffic goes to the internet (Data Transfer Out - DTO)

Additionally, this Notebook provides the following CSV files:

1. Recommended instance types, with EBS data, DTO, RAM, IOPs and vCPUs for each instance
2. VMs that couldn't be processed by AWS ADS
3. EC2 instances with no activity (0 disk IOPs)
4. Instances with less than 10 disk IOPs
5. Top ten instances, in terms of cost

Please modify following variables as per your file names:

In [ ]:
# Cell 1
# Importing needed libraries
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
import json


# Please replace with the name of the files obtained from AWS ADS
ec2_recommendation_file = "EC2InstanceRecommendations.csv"
system_performance_file = "SystemPerformance.csv"


In this section we query the EBS prices using Pricing API ( source: https://aws.amazon.com/ebs/pricing ). Please change the region as needed:

In [ ]:
# Cell 2
print("Get EBS Volume Prices")
print("==============================")

pricing = boto3.client('pricing')

#Change 'location' below if needed:
response = pricing.get_products(
     ServiceCode='AmazonEC2',
     Filters = [
         {'Type' :'TERM_MATCH', 'Field':'volumeApiName', 'Value':'standard'              }
         ,{'Type' :'TERM_MATCH', 'Field':'locationType',  'Value':'AWS Region'                   }
         ,{'Type' :'TERM_MATCH', 'Field':'location',      'Value':'US East (N. Virginia)'              }
     ],
     MaxResults=100
)


# From the whole response get the list of key names (product codes) that we are intereste on,
# which are the OnDemand Price Dimensions. Each range of price has a unique code name that we
# don't know in advance. That's why we have to get the keys list to iterate them later.
product = json.loads(response['PriceList'][0])
ondem = product['terms']['OnDemand']
ondem_list = list(ondem.keys())
prices = ondem[ondem_list[0]]['priceDimensions']
prices_list = list(prices.keys())

# Based on the top GBs of the range determine the cost variable of interest
for range_code in prices_list:
    range = prices[range_code] 
    if range['endRange'] == 'Inf':
        ebs_magnetic_monthly_cost  = float(range['pricePerUnit']['USD'])

print('ebs_magnetic_monthly_cost =  $ {} USD'.format(ebs_magnetic_monthly_cost))

##########################################

#Change 'location' below if needed:
response = pricing.get_products(
     ServiceCode='AmazonEC2',
     Filters = [
         {'Type' :'TERM_MATCH', 'Field':'volumeApiName', 'Value':'gp3'              }
         ,{'Type' :'TERM_MATCH', 'Field':'locationType',  'Value':'AWS Region'                   }
         ,{'Type' :'TERM_MATCH', 'Field':'location',      'Value':'US East (N. Virginia)'              }
     ],
     MaxResults=100
)

# From the whole response we search for the specific "usagetype" we are looking for between all prices.
# From thta product we get the list of key names (product codes) that we are intereste on,
# which are the OnDemand Price Dimensions. Each range of price has a unique code name that we
# don't know in advance. That's why we have to get the keys list to iterate them later.
pricelist = response['PriceList']
for i in pricelist:
    tmp_prod = json.loads(i)
    if tmp_prod['product']['attributes']['usagetype'] == 'EBS:VolumeUsage.gp3':
        product = tmp_prod

ondem = product['terms']['OnDemand']
ondem_list = list(ondem.keys())
prices = ondem[ondem_list[0]]['priceDimensions']
prices_list = list(prices.keys())

# Based on the top GBs of the range determine the cost variable of interest
for range_code in prices_list:
    range = prices[range_code]
    if range['endRange'] == 'Inf':
        ebs_gp3_monthly_cost   = float(range['pricePerUnit']['USD'])

print('ebs_gp3_monthly_cost  = $ {} USD'.format(ebs_gp3_monthly_cost ))

In this section we query the DTO prices using Pricing API ( source: https://aws.amazon.com/ec2/pricing/on-demand/ ). Please change the region as needed:

In [ ]:
# Cell 3
print("Get DataTransfer Prices")
print("==============================")

# Change 'location' below if needed:
response = pricing.get_products(
     ServiceCode='AWSDataTransfer',
     Filters = [
         {'Type' :'TERM_MATCH', 'Field':'fromLocationType', 'Value':'AWS Region'              }
         ,{'Type' :'TERM_MATCH', 'Field':'transferType',  'Value':'AWS Outbound'                   }
         ,{'Type' :'TERM_MATCH', 'Field':'fromLocation',      'Value':'US East (N. Virginia)'              }
     ],
     MaxResults=100
)

# From the whole response get the list of key names (product codes) that we are intereste on,
# which are the OnDemand Price Dimensions. Each range of price has a unique code name that we
# don't know in advance. That's why we have to get the keys list to iterate them later.
product = json.loads(response['PriceList'][0])
ondem = product['terms']['OnDemand']
ondem_list = list(ondem.keys())
prices = ondem[ondem_list[0]]['priceDimensions']
prices_list = list(prices.keys())

# Based on the top GBs of the range determine the cost variable of interest
for range_code in prices_list:
    range = prices[range_code]
    
    if range['endRange'] == '10240':
        cost_10_tb = float(range['pricePerUnit']['USD'])
    elif range['endRange'] == '51200':
        cost_40_tb = float(range['pricePerUnit']['USD'])
    elif range['endRange'] == '153600':
        cost_100_tb = float(range['pricePerUnit']['USD'])
    elif range['endRange'] == 'Inf':
        cost_150_tb = float(range['pricePerUnit']['USD'])

print('cost_10_tb = $ {} USD'.format(cost_10_tb))
print('cost_40_tb = $ {} USD'.format(cost_40_tb))
print('cost_100_tb = $ {} USD'.format(cost_100_tb))
print('cost_150_tb = $ {} USD'.format(cost_150_tb))

## 2. Data Preparation

In [ ]:
# Cell 4
# Importing the recommendations file
ds_rec = pd.read_csv(ec2_recommendation_file)

In [ ]:
# Cell 5
# Importing the performance file
ds_sys = pd.read_csv(system_performance_file)

In [ ]:
# Cell 6
# Copying to a dataframe with the needed columns from recommendations dataframe
ds_rec_fil = ds_rec[['ServerId','Server.HostName','Server.VMware.VMname','Server.OS.Name','Recommendation.EC2.Instance.OSType','Server.VMware.vCenterName','Recommendation.EC2.Instance.vCPUCount','Recommendation.EC2.Instance.RAM.TotalSizeinMB','Recommendation.EC2.Instance.Model','Recommendation.EC2.Instance.Price.HourlyRate','Server.DiskReadOpsPerSecond.Max','Server.DiskWriteOpsPerSecond.Max']].copy()

In [ ]:
# Cell 7
# Renaming the cells to something easier to identify
ds_rec_fil.rename(columns={'ServerId': 'Server_ID','Server.HostName': 'Server','Server.VMware.VMname':'Vmware_Name','Server.OS.Name':'Operating_System','Recommendation.EC2.Instance.OSType':'OS_Type','Server.VMware.vCenterName':'vCenter_Name','Recommendation.EC2.Instance.vCPUCount':'vCPUs_Recommendation','Recommendation.EC2.Instance.RAM.TotalSizeinMB':'Memory_(GiB)_Recommendation','Recommendation.EC2.Instance.Model':'Recommended_EC2_Instance','Recommendation.EC2.Instance.Price.HourlyRate':'EC2_Hourly_Rate','Server.DiskReadOpsPerSecond.Max':'DiskReadOpsPerSecondMax','Server.DiskWriteOpsPerSecond.Max':'DiskWriteOpsPerSecondMax'},inplace=True)

In [ ]:
# Cell 8
# Showing 10 random rows
np.random.seed(1)
ds_rec_fil.sample(n=10)

In [ ]:
# Cell 9
# Copying to a dataframe with the needed columns from performance dataframe
ds_sys_fil = ds_sys[['serverId','numDisks','powerState','totalDiskSize','avgNetworkBytesWrittenPerSecond']].copy()

In [ ]:
# Cell 10
# Renaming the cells to something easier to identify
ds_sys_fil.rename(columns={'serverId': 'Server_ID', 'numDisks': 'Num_Disks', 'powerState': 'VM_State(ON/OFF)','totalDiskSize':'Storage_Capacity(GiB)', 'avgNetworkBytesWrittenPerSecond': 'Data_Transfer_Out-GiB_written'}, inplace=True)

In [ ]:
# Cell 11
# Showing 10 random rows
ds_sys_fil.sample(n=10)

In [ ]:
# Cell 12
# Showing the data type for each column in the recommendations dataframe
ds_rec_fil.info()

In [ ]:
# Cell 13
# Showing the data type for each column in the performance dataframe
ds_sys_fil.info()

In [ ]:
# Cell 14
# Joining both data frames (inner join)
df_merged = ds_rec_fil.merge(ds_sys_fil, on='Server_ID', how='inner')

In [ ]:
# Cell 15
# Showing 10 random rows
df_merged.sample(n=10)

In [ ]:
# Cell 16
# Functions to transform MiB to GiB and Bytes to GiB
def mb2gb(mb):
    return mb/1024

def b2gb(b):
    if np.isnan(b) == False:
        return round(b/1024/1024/1024)

# Function to compute the DTO consumed during a month, from the bits written to the network
def monthly(b):
    if np.isnan(b) == False:
        return round(b*3600*24*30.4)

# Function to transform bits to GiB
def bi2gb(b):
    if np.isnan(b) == False:
        return round(b/8/1024/1024/1024)

In [ ]:
# Cell 17
# Transforming Memory data to GiB
df_merged['Memory_(GiB)_Recommendation'] = df_merged['Memory_(GiB)_Recommendation'].apply(mb2gb)

In [ ]:
# Cell 18
# Transforming Memory data to GiB
df_merged['Data_Transfer_Out-GiB_written'] = df_merged['Data_Transfer_Out-GiB_written'].apply(monthly)
df_merged['Data_Transfer_Out-GiB_written'] = df_merged['Data_Transfer_Out-GiB_written'].apply(b2gb)

In [ ]:
# Cell 19
# Transforming EBS storage data to GiB
df_merged['Storage_Capacity(GiB)'] = df_merged['Storage_Capacity(GiB)'].apply(b2gb)

In [ ]:
# Cell 20
# Showing 10 random rows
df_merged.sample(n=10)

In [ ]:
# Cell 21
df_merged.info()

### Insights:
* If we have more `Server_ID`, w.r.t. `Recommended_EC2_Instance`, it's possible that AWS ADS was not able collect data because the VM was turned off.
* We will filter out turned off VMs


In [ ]:
# Cell 22
# Selecting turned on VMs only
df_on = df_merged[df_merged['VM_State(ON/OFF)'] == 'POWER_ON']
df_on.info()

### Insights:
* If event after running previous code, we see that the `Server_ID` non-null > `Recommended_EC2_Instance` non-null, we will need to copy those VMs for which ADS does not have a recommendation.
* We have observed it's normal to have more `Server_ID` than `Recommended_EC2_Instance`, mainly because:

1. ESX servers
2. VMware appliances that we don't need to migrate
3. The AWS ADS Connector appliance or any other AWS appliances, such as AWS Storage Gateway
4. Instances that don't have VMware Tools installed.

We export all VMs falling into "Non_Recommandation" case, for its manual treatment, using other tools like, for example, RVTools.

In [ ]:
# Cell 23
# Getting instances without recommendation
df_no_recom = df_on[df_on['Recommended_EC2_Instance'].isnull()]

In [ ]:
# Cell 24
# Saving the instances without recommendation in a CSV file for its further analysis, outside the Notebook
df_no_recom.to_csv("EC2_Non_Recommandation.csv")

### Removing Null values

We will select only values for which ADS has a recommendation, prior Exploratory Data Analysis.

In [ ]:
# Cell 25
# Counting the rows and columnds
df_on_rec = df_on[df_on['Recommended_EC2_Instance'].notnull()]

In [ ]:
# Cell 26
df_on_rec.info()

In [ ]:
# Cell 27
# Adding write and read IOPs
df_on_rec['Iops_total_(R+W)-Max'] = df_on_rec['DiskReadOpsPerSecondMax'] + df_on_rec['DiskWriteOpsPerSecondMax']


In [ ]:
# Cell 28
df_on_rec.head()

### Optimizing EBS volume types

We assume that if the IOPs number is below 100, it's possible to use a previous generation volume in our application: https://aws.amazon.com/ebs/previous-generation/

However, it's needed to validate case by case if our application is going to work correctly with a magnetic volume.

In [ ]:
# Cell 29
# Functions to determine the EBS type and its price, taking into account the IOPs and the disk size. If disk size > 1024 GiB, they need to be GP3 at least

def ebs_type_iops(n):
    if n > 100:
        return 1.0
    else:
        return 0

def ebs_type_gb(n):
    if n > 1024:
        return 1.0
    else:
        return 0

def ebs_type_encode(n):
    if n >= 1:
        return 'gp3'
    else:
        return 'magnetic'

def ebs_price_encode(n):
    if n >= 1:
        return ebs_gp3_monthly_cost
    else:
        return ebs_magnetic_monthly_cost

In [ ]:
# Cell 30
df_on_rec['EBS_Type_Iops'] = df_on_rec['Iops_total_(R+W)-Max'].apply(ebs_type_iops)
df_on_rec['EBS_Type_Gb'] = df_on_rec['Storage_Capacity(GiB)'].apply(ebs_type_gb)
df_on_rec['EBS_Type_Gb+Iops'] = df_on_rec['EBS_Type_Iops'] + df_on_rec['EBS_Type_Gb']
df_on_rec['EBS_Type'] = df_on_rec['EBS_Type_Gb+Iops'].apply(ebs_type_encode)
df_on_rec['EBS_Price'] = df_on_rec['EBS_Type_Gb+Iops'].apply(ebs_price_encode)
# Computing the monthly cost per volume
df_on_rec['EBS_Monthly_Cost'] = df_on_rec['EBS_Price'] * df_on_rec['Storage_Capacity(GiB)']
# Computing the monthly cost per instance per month, from the hourly rate from each instance
df_on_rec['EC2_Monthly_Cost'] = df_on_rec['EC2_Hourly_Rate'] * 730
df_on_rec.drop(['EBS_Type_Iops', 'EBS_Type_Gb', 'EBS_Type_Gb+Iops'], axis=1, inplace=True)

In [ ]:
# Cell 31
df_on_rec.sample(n=10)

## 3. Exploratory Data Analysis

With collected data, we draw some graphs that will let us analysing visually the type of recommended instances, the operating systems and the Windows versions distribution.

Additionally, we select the list of instances with less than 10 IOPs, which can help us identify low or null utilization VMs and determine if we need to migrate them to AWS.


In [ ]:
# Cell 32
# Function to graph interesting columns
def chart_category(data_column, width, height, add_percentage=False):
    plt.figure(figsize=(width,height))
    chart = sns.countplot(x=data_column, order = data_column.value_counts().index)
    chart.set_xticklabels(chart.get_xticklabels(),rotation=45)
    
    for p in chart.patches:
        total = len(data_column)
        if add_percentage == False:
            percentage = p.get_height()
        else:
            percentage = '{:.1f}%'.format(100 * p.get_height()/total) + '\n(' + str(p.get_height()) + ')'
        x = p.get_x() + p.get_width() / 2 
        y = p.get_y() + p.get_height() + 0.1
        chart.annotate(percentage, (x, y), size = 10)

In [ ]:
# Cell 33
df_on_rec.info()

In [ ]:
# Cell 34
# We graph the recommended instance distribution
# This will be useful to understand the final costs of EC2, when comparing different scenarios
chart_category(df_on_rec['Recommended_EC2_Instance'], 20, 10, True)

In [ ]:
# Cell 35
# Graphing other interesting columns such as OS type and windows versions
interesting_columns = ['OS_Type', 'EBS_Type']
for column in interesting_columns:
    chart_category(df_on_rec[column], 10, 5, True)

In [ ]:
# Cell 36
# We will analyse the Windows version distribution
df_on_rec_win = df_on_rec[df_on_rec['OS_Type']=='Windows']
chart_category(df_on_rec_win['Operating_System'], 20, 10, True)

## VMs with 0 disk IOPs and with < 10 disk IOPs

In this section we provide the list of VMs that didn't show disk IOPs activity during ADS collection. Those VMs could be removed from the migration project to save costs in AWS.

In [ ]:
# Cell 37
# To have consistency with Migration Evaluator, we will call the VMs with 0 disk IOPs as zombie VMs
df_zombie = df_on_rec[df_on_rec["Iops_total_(R+W)-Max"] == 0]

In [ ]:
# Cell 38
# We save Zombies VMs in a CSV
df_zombie.to_csv('Zombie_vm.csv')

In [ ]:
# Cell 39
# To have consistency with Migration Evaluator, we will call the VMs with 10 disk IOPs or less, as zombie VMs
df_zombie_ten_iops = df_on_rec[df_on_rec["Iops_total_(R+W)-Max"] < 10]

In [ ]:
# Cell 40
# We save Zombies VMs in a CSV
df_zombie_ten_iops.to_csv('Zombie_vm_less_than_10_iops.csv')

## Top 10 instances, by cost
We provide the Top 10 of the insances, by cost. We provide this list for further analysis, like for example, selecting a different OS or instance type.

In [ ]:
# Cell 41
ds_top10 = df_on_rec.sort_values('EC2_Monthly_Cost', ascending=False).head(10)

In [ ]:
# Cell 42
ds_top10.to_csv('top_10.csv', index=False)

# 4. Cost Analysis and Recommendations Export

At the end, we are interested on what is going to be the infrastructure cost on AWS (EC2, EBS and DT0). In this section we provide a summary of those coses.
It's recommended to evaluate different scenarios, such as Direct Match, and 95 Percentile, to determine which one is more convenient for the business.

## EC2 Costs

In [ ]:
# Cell 43
ec2_cost = df_on_rec['EC2_Monthly_Cost'].sum().round(2)
print(f"The total EC2 Monthly Cost would be $ {ec2_cost} USD")

## EBS Costs

In [ ]:
# Cell 44
ebs_cost = df_on_rec['EBS_Monthly_Cost'].sum().round(2)
print(f"The total EBS Monthly Cost would be $ {ebs_cost} USD")

## DTO Cost Estimation

In [ ]:
# Cell 45
total_gb_dto = df_on_rec['Data_Transfer_Out-GiB_written'].sum().round(2)*0.2

dto_monthly_cost = 0
if total_gb_dto < 10*1024:
    dto_monthly_cost = total_gb_dto * cost_10_tb
elif total_gb_dto < 50*1024:
    dto_monthly_cost = 10*1024*cost_10_tb + (total_gb_dto - 10*1024)*cost_40_tb
elif total_gb_dto < 150*1024:
    dto_monthly_cost = 10*1024*cost_10_tb + 40*1024*cost_40_tb + (total_gb_dto - 50*1024)*cost_100_tb
else:
    dto_monthly_cost = 10*1024*cost_10_tb + 40*1024*cost_40_tb + 100*1024*cost_100_tb + (total_gb_dto - 150*1024)*cost_150_tb
    
print(f"The total DTO Monthly Cost would be $ {dto_monthly_cost.round(2)} USD")

## Total Cost of Ownership (TCO)

In [ ]:
# Cell 46
print(f"The total estimated TCO is $ {ec2_cost + ebs_cost + dto_monthly_cost.round(2)} USD (Monthly)")

## Export recommendation file

Finally, we export the recommendation file with the recommended instances and storage per VM. This is useful to do a bulk import into the AWS Calculator https://calculator.aws/ and try different purchase options, such as Compute Savings Plans.

In [ ]:
# Cell 47
# Exporting the recommendation file
df_on_rec.to_csv(ec2_recommendation_file.split('.')[0] + "_output.csv", index=False)

# ------------ END